<a href="https://colab.research.google.com/github/fabiooliveir/facebook-leads-api/blob/main/Consulta_API_de_Cadastros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import os

## Consulta para Obtenção do Access Token da Página do Facebook

Nesta célula, realizamos uma consulta à API do Facebook para obter o Access Token da página. Este token é necessário para realizar operações em nome da página, como publicar conteúdo ou obter métricas. Certifique-se de substituir 'ID_DA_PAGINA' pelo ID da sua página e '[[TOKEN USUÁRIO]]' pelo seu token de usuário válido.

### Parâmetros:
- `url1`: URL da API do Facebook para obter o Access Token.
- `params1`: Parâmetros da consulta, incluindo os campos a serem retornados ('access_token') e o token de acesso do usuário.
- `response1`: Resposta da API contendo o Access Token.
- `access_token`: Access Token da página, utilizado posteriormente para realizar operações na página do Facebook.


In [ ]:
url1 = "https://graph.facebook.com/v19.0/"+os.environ.get('ID_DA_PAGINA')
params1 = {
    'fields': 'access_token',
    'access_token': '[[TOKEN USUÁRIO]]'
}
response1 = requests.get(url1, params=params1)
access_token = response1.json()['access_token']


## Consulta para Obtenção de Leads das Formulários de Geração de Leads

Nesta consulta, utilizamos o Access Token obtido anteriormente para acessar os formulários de geração de leads da página do Facebook. Estamos solicitando os leads gerados por esses formulários.

### Parâmetros:
- `url2`: URL da API do Facebook para obter os leads dos formulários de geração de leads da página.
- `params2`: Parâmetros da consulta, incluindo os campos a serem retornados ('access_token,leadgen_forms{leads}') e o Access Token da página.
- `response2`: Resposta da API contendo os leads dos formulários de geração de leads.


In [ ]:

url2 = "https://graph.facebook.com/v19.0/"+os.environ.get('ID_DA_PAGINA')
params2 = {
    'fields': 'access_token,leadgen_forms{leads}',
    'access_token': access_token
}
response2 = requests.get(url2, params=params2)

print(response2.json())

## Extração e Processamento de Dados dos Leads

Nesta célula, extraímos os dados dos leads dos formulários de geração de leads da página do Facebook. O objeto `response2.json()` contém a resposta da API do Facebook, que é um dicionário contendo os leads dos formulários. Em seguida, iteramos sobre os leads para extrair os dados necessários, como nome completo, e-mail e número de telefone.

### Passos:
1. Extrair os leads da resposta da API.
2. Iterar sobre os leads e extrair os dados de cada lead.
3. Criar um DataFrame do Pandas com os dados extraídos.

### Variáveis:
- `leads`: Lista de leads dos formulários de geração de leads.
- `full_names`: Lista de nomes completos dos leads.
- `emails`: Lista de e-mails dos leads.
- `phone_numbers`: Lista de números de telefone dos leads.
- `lead_data`: Dados de um lead específico.
- `lead_dict`: Dicionário contendo os dados de um lead.
- `df`: DataFrame do Pandas contendo os dados extraídos.


In [ ]:
response_data = response2.json()

leads = response_data.get('leadgen_forms', {}).get('data', [])

full_names = []
emails = []
phone_numbers = []

for lead in leads:
    lead_data = lead.get('leads', {}).get('data', [])
    for data in lead_data:
        if 'field_data' in data:
            lead_dict = {item['name']: item['values'][0] for item in data['field_data']}
            full_names.append(lead_dict.get('full_name', None))
            emails.append(lead_dict.get('email', None))
            phone_numbers.append(lead_dict.get('phone_number', None))
        else:
            full_names.append(None)
            emails.append(None)
            phone_numbers.append(None)

df = pd.DataFrame({
    'full_name': full_names,
    'email': emails,
    'phone_number': phone_numbers
})

df.head()